# TODO：概要



# Connected Agents の主な制限

### 1. **ローカル関数は呼び出せない**

* **制限内容**
  　Connected Agents 機能を使って別のエージェントをツールとして追加した場合、その**サブエージェント側で定義した関数（Pythonのローカル関数）をFunction Tool経由で直接呼び出すことはできません**。
* **推奨される方法**
  　どうしても外部の処理を呼び出したい場合は、「OpenAPIツール」や「Azure Functions」を使って、API経由で実装することが推奨されています。

### 2. **出典（引用）の引き継ぎが保証されない**

* **制限内容**
  　Connected Agents経由で検索や回答をした際、**サブエージェントが生成した引用情報（例えばドキュメントの出典や根拠）が、親エージェントに必ず渡るとは限りません**。
* **対応策（完全ではない）**
  　プロンプトの工夫（Prompt Engineering）や、使うAIモデルを変えることで出典が返りやすくする方法もありますが、**必ず引用が親エージェントに引き継がれる保証はありません**。

### まとめ

* サブエージェントの**ローカル関数は直接使えない**ので、API公開（OpenAPIやAzure Functions）を利用しましょう。
* サブエージェントの**引用情報が必ず親エージェントまで引き継がれるわけではない**点も注意が必要です。

参考：[Limitations](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/connected-agents?pivots=python#limitations)

# ライブラリのインポート

In [59]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
    FilePurpose,
    FileSearchTool,
    RunAdditionalFieldList,
    RunStepFileSearchToolCall,
    RunStepToolCallDetails,
    ConnectedAgentTool
)


# 環境変数の取得

In [117]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID=os.getenv("FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID")
FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")
FOUNDRY_MCP_AGENT_ID=os.getenv("FOUNDRY_MCP_AGENT_ID")
FOUNDRY_FILE_SEARCH_AGENT_ID=os.getenv("FOUNDRY_FILE_SEARCH_AGENT_ID")

# クライアントの初期化

In [118]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# AgentClient の作成
agents_client = project_client.agents

# ユーティリティ関数

In [119]:
def agent_run_outputs(thread_id, agents_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

In [120]:
def file_search_content_outputs(agents_client, thread_id, run_id):
    """
    指定したスレッドID・Run IDのファイル検索ツールによる検索結果コンテンツをNotebook等に出力します。

    :param agents_client: AgentClient インスタンス
    :param thread_id: スレッドID
    :param run_id: Run ID
    """
    from azure.ai.agents.models import RunAdditionalFieldList, RunStepToolCallDetails, RunStepFileSearchToolCall

    for run_step in agents_client.run_steps.list(
        thread_id=thread_id,
        run_id=run_id,
        include=[RunAdditionalFieldList.FILE_SEARCH_CONTENTS]
    ):
        if isinstance(run_step.step_details, RunStepToolCallDetails):
            for tool_call in run_step.step_details.tool_calls:
                if (
                    isinstance(tool_call, RunStepFileSearchToolCall)
                    and getattr(tool_call, "file_search", None)
                    and getattr(tool_call.file_search, "results", None)
                    and tool_call.file_search.results
                    and getattr(tool_call.file_search.results[0], "content", None)
                    and tool_call.file_search.results[0].content
                    and getattr(tool_call.file_search.results[0].content[0], "text", None)
                ):
                    file_name = tool_call.file_search.results[0].file_name
                    content_text = tool_call.file_search.results[0].content[0].text
                    print(f"ファイル検索の結果、以下のコンテンツがヒットしました。\n「{file_name}」:")
                    print("=" * 60)
                    print(content_text)
                    print("=" * 60)

# Connected Agents の定義

In [121]:
consulting_agent  = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="consulting_agent ",
    instructions=(
        "あなたは資産運用のコンサルタントとして、ユーザーの目的や希望、リスク許容度、資産状況などをヒアリングし、"
        "最適な運用方針のアドバイスや意思決定のサポートを行います。"
        "必要に応じて「目標金額」「運用期間」「現在の資産額」「リスク許容度」など、計算・シミュレーションに必要な情報を丁寧に聞き出してください。" 
        "計算やシミュレーションが必要な場合は、具体的な内容をcode_interpreter_agentに依頼するよう促してください。"
        "ユーザーが迷っている場合は、選択肢やリスク・メリットをわかりやすく提示してください。"
    ),
)
print(f"Created Agent. AGENT_ID: {consulting_agent.id}")

Created Agent. AGENT_ID: asst_tJuHK2CcoGyxhagNVYDPi1yf


In [131]:
# エージェントの取得
code_interpreter_agent = agents_client.get_agent(agent_id=FOUNDRY_CODE_INTERPRETER_AGENT_ID)
file_search_agent = agents_client.get_agent(agent_id=FOUNDRY_FILE_SEARCH_AGENT_ID)
mcp_agent = agents_client.get_agent(agent_id=FOUNDRY_MCP_AGENT_ID)

# ConnectedAgentTool の作成
consulting_agent_tool = ConnectedAgentTool(
    id=consulting_agent.id,
    name="consulting_agent",
    description=(
        "Consulting Agent を利用して、資産運用のコンサルティングを行います。"
        "ユーザーの目的や希望、リスク許容度、資産状況などをヒアリングし、最適な運用方針のアドバイスや意思決定のサポートを行います。"
    ),
)

code_interpreter_agent_tool = ConnectedAgentTool(
    id=code_interpreter_agent.id,
    name="code_interpreter_agent",
    description=(
        "Code Interpreter を利用して、計算や図表の出力に特化した分析アシスタントです。"
    ),
)

file_search_agent_tool = ConnectedAgentTool(
    id=file_search_agent.id,
    name="file_search_agent",
    description=(
        "File Search Agent を利用して、ファイルの検索や内容の取得を行うことができます。"
    ),
)

# エージェントの作成

In [134]:
triage_agent  = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="triage_agent ",
    # instructions=(
    #     "あなたは「資産運用アシスタント」として、ユーザーの質問や相談を理解し、適切な専門エージェントにタスクを割り振ります。"
    #     "はじめにconsulting_agentにユーザーからの質問・相談内容を分析してもらい、"
    #     "次に、計算やシミュレーションが必要な場合はcode_interpreter_agentにタスクを依頼してください。"
    # ),
    instructions=(
        "あなたは振り分けエージェントです。ユーザーの質問にあわせて適切な専門エージェントにタスクを割り振ります。"
        " - file_search_agent: カスタマーサポート担当で、社内のドキュメントを検索し、ユーザーの質問に答えます。"
        " - code_interpreter_agent: 計算や図表の出力に特化した分析アシスタントです。"
    ),
    tools=[
        code_interpreter_agent_tool.definitions[0],
        file_search_agent_tool.definitions[0],
        # consulting_agent_tool.definitions[0],
    ],
    # temperature=0.1
)
print(f"Created Agent. AGENT_ID: {triage_agent.id}")


Created Agent. AGENT_ID: asst_rgRuVuvqhvervNkvbJT2ZVPo


In [125]:
# triage_agent_dict = triage_agent.as_dict()
# print(json.dumps(triage_agent_dict, indent=2, ensure_ascii=False))

# スレッドの作成

In [143]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_KJUCOwV4GCkW3cluExyj67WP


# ユーザーメッセージの追加

In [136]:
# メッセージの追加
# user_message= (
#     "初期投資額が100万円、年利回りが3.5%、毎年10万円ずつ追加投資を"
#     "20年間継続したときの最終的な資産額を計算し、円単位で四捨五入して答えてください。"
# )

# user_message = (
#     "現在の貯金が300万円あり、5年後にマイホームの頭金として1000万円を目標にしています。"
#     "毎月いくら積み立てれば、年利3%で達成できますか？また、無理のない積立額や運用方針もアドバイスしてください。"
# )

user_message = (
    "新しいパソコンの購入を検討しています。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。"
)

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_afOI4PUsjkMu4VMEs8EOzBEH


# Run の実行

In [137]:
run_1 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=triage_agent.id,
)

if run_1.status == "failed":
    print(f"Run failed: {run_1.last_error}")
else:
    agent_run_outputs(thread.id, agents_client)

USER: 新しいパソコンの購入を検討しています。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。
ASSISTANT: パソコンの初期設定サポートについて、次のようなサービスが用意されています。

---

### 🎉 初心者向けセットアップサポート（所要時間：約30〜60分） 
- 電源の接続と起動確認
- Wi-Fi・インターネット接続設定
- メールアカウント・アプリの初期登録
- セキュリティ対策（ウイルス対策ソフトの確認・導入）
- 基本操作レクチャー（マウス、キーボード、タッチ操作）
- よく使うアプリのインストールサポート（Office・Zoomなど）

### サポートの特徴
- 出張対応・オンライン対応が選べます
- 初回サポート無料（キャンペーン中）
- ご希望に応じてマニュアルもプレゼント

### お問い合わせ・ご予約
- 電話：0120-123-456（平日 10:00〜18:00）
- メール：support@easysetup.jp
- WEB予約：https://www.easysetup.jp

---

不安な点や詳しい内容、ご予約については上記の連絡先までお気軽にご相談ください。


In [ ]:
# 参照したコンテンツを出力
file_search_content_outputs(agents_client, thread.id, run_1.id)

In [144]:
user_message = (
    "年利4%で、毎月6万円を積み立てた場合、20年後の資産額を計算してグラフを出力してください。"
)

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

run_2 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=triage_agent.id,
)

if run_2.status == "failed":
    print(f"Run failed: {run_2.last_error}")
else:
    agent_run_outputs(thread.id, agents_client)

USER: 年利4%で、毎月6万円を積み立てた場合、20年後の資産額を計算してグラフを出力してください。
ASSISTANT: タスクを処理中にエラーが発生しました。もう一度試してよろしいでしょうか？それとも、他のご質問やご要望がありますか？


In [129]:
user_message = (
    "リスク許容度は「中程度」で、資産運用の目的は「将来の教育資金の準備」です。"
    "月5万円の積立までならいけます。"
)


message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=triage_agent.id,
)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    agent_run_outputs(thread.id, agents_client)

USER: 現在の貯金が300万円あり、5年後にマイホームの頭金として1000万円を目標にしています。毎月いくら積み立てれば、年利3%で達成できますか？また、無理のない積立額や運用方針もアドバイスしてください。
ASSISTANT: シミュレーション計算に進む前に、より適切なアドバイスを行うため、以下の2点についてご回答をお願いいたします。

1. **リスク許容度**  
年利3％を目指す場合、預金よりはリスクのある商品（例：投資信託、債券など）も含まれます。  
- 「元本割れはしたくない」  
- 「多少の変動は許容」  
- 「リターン重視である程度リスクも取る」  
など、ざっくりしたイメージで教えてください。

2. **毎月の無理のない積立可能額**  
（例：「月5万円なら大丈夫」「今の貯蓄ペースで計算してほしい」など）

ご協力いただけますと、積立額の目安や最適な運用方法のご提案がより具体的になります。よろしくお願いします！

それでは、ご回答をお待ちしております。
USER: リスク許容度は「中程度」で、資産運用の目的は「将来の教育資金の準備」です。月5万円の積立までならいけます。
ASSISTANT: シミュレーション結果をまとめます。

【条件】
- 初期貯金：300万円
- 目標金額：1000万円（5年後）
- 年利：3%
- リスク許容度：中程度
- 毎月積立：最大5万円

---

## 結果

1. **5年後に目標達成のために必要な毎月積立額**
   - 約「100,992円」必要です。現状の積立上限（月5万円）では、目標金額に届きません。

2. **月5万円ずつ積み立てた場合の5年後の到達予想額**
   - 約「6,706,870円」となり、目標（1000万円）の約67%となります。

---

## アドバイス

- 月5万円の積立と年利3％の運用を継続した場合、現状の条件だと5年後の時点では約670万円まで資金を準備できます。
- 目標（1000万円）との差額約330万円を埋めるためには、
    1. 積立額の増額
    2. 運用期間の延長（例えば6～7年に伸ばす等）
    3. より高いリターン商品に一部チャレンジ
    いずれかの対応が必要です。
- リスク許容は「中程度」とのことですので、つみたてNISA

# Agent ID を .env ファイルに保存
※ 今回作成したエージェントを、後続の Connected Agents のハンズオン演習で使用するため永続化します。

In [ ]:
# 変数の定義
agent_env_key = "FOUNDRY_TRIAGE_AGENT_ID"
agent_env_value = triage_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')
